# 300_gpt_tuning_v2

> In this notebook we will fine tune multiple GPT 3.5 models with what we learned from the first set and with progressive and conservative models seperate.

In [132]:
import pandas as pd
import numpy as np
import seaborn as sb
import matplotlib.pyplot as plt
import openai
import json
from googleapiclient import discovery
from googleapiclient.errors import HttpError


Enable apis with keys. 

In [133]:
with open("C:/Users/danie/OneDrive/Desktop/buffed_perspective_api_key_.txt") as f:
    papi_key1 = f.readline()
    
with open("C:/Users/danie/OneDrive/Desktop/perspective_api_key.txt") as f:
    papi_key2 = f.readline()
    

perspective_client0 = discovery.build(
  "commentanalyzer",
  "v1alpha1",
  developerKey=papi_key1,
  discoveryServiceUrl="https://commentanalyzer.googleapis.com/$discovery/rest?version=v1alpha1",
  static_discovery=False,
)

perspective_client1 = discovery.build(
  "commentanalyzer",
  "v1alpha1",
  developerKey=papi_key2,
  discoveryServiceUrl="https://commentanalyzer.googleapis.com/$discovery/rest?version=v1alpha1",
  static_discovery=False,
)


with open("C:/Users/danie/OneDrive/Desktop/openai_youtube_api_key.txt") as f:
    api_key = f.readline()

openai.api_key = api_key

Set up helper functions:

In [134]:
# Quantifier 
def get_toxicity_score(comment, client):
    analyze_request = {
        'comment': { 'text': comment },
        'languages': ["en"],
        'requestedAttributes': {'TOXICITY': {}}
    }
    
    try:
        response = eval(client).comments().analyze(body=analyze_request).execute()
        return float(response['attributeScores']['TOXICITY']['summaryScore']['value'])
    except HttpError as e:
        return [e.resp.status]

In [135]:
import time
def get_toxicity_score_wrapper(inp):
    if type(inp) == type({}):
        content = inp
    elif type(inp) == type([]):
        if type(inp[0]) == type({}):
            content = [l['messages'][2]['content'] for l in inp]
    else:
        return "Invalid input type!"

    print("Beginning Toxcity Assessment:")
    toxicity_vals = []
    switch_count = 0
    total_lines = len(content)
    current_client = f"perspective_client{switch_count}"
    for i, l in enumerate(content):
        s = get_toxicity_score(l, current_client)
        if s == [429]:
            print("Switched Perspective Clients!")
            switch_count +=1 
            current_client = f"perspective_client{switch_count % 2}"
            s = get_toxicity_score(l, current_client)
        if type(s) != type(0.0):
            return f"Error! S = {s}"
        else:
            toxicity_vals.append(s)
        time.sleep(1)
        percentage_complete = (i+1)/total_lines * 100
        if percentage_complete % 1 == 0:
            print(f"{percentage_complete}% Processed.")

    return toxicity_vals
    

In [136]:
def format_comment_for_finetuning(row):
    affil = row.affiliation
    affil = 'conservative' if affil == 'R' else 'progressive'
    title = row.video_title
    desc = row.video_description
    comment = row.comment_text
    formatted = {"messages": [{"role": "system", "content": f"You are a {affil} American political commentator."},
                              {"role": "user", "content": f"You just watched the youtube video '{title}' with the description '{desc}'.\n\nGive your opinion on the subject matter."},
                              {"role": "assistant", "content": comment}]}
    return json.dumps(formatted)

In [137]:
comments_df = pd.read_csv("../data/cleaned/channel_subset_with_comments.csv", index_col='comment_id')
comments_df.columns

Index(['channel_id', 'channel_title', 'affiliation', 'video_id', 'video_title',
       'video_description', 'comment_text'],
      dtype='object')

In [138]:
# Right data will be from Turning Point USA
right_comments = comments_df.loc[comments_df.affiliation == 'R']
right_comments.reset_index(drop=True, inplace=True)
# Left comments will be from The Young Turks 
left_comments =comments_df.loc[comments_df.affiliation == "L"]
left_comments.reset_index(drop=True, inplace=True)

print(right_comments.info())
print(left_comments.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 33762 entries, 0 to 33761
Data columns (total 7 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   channel_id         33762 non-null  object
 1   channel_title      33762 non-null  object
 2   affiliation        33762 non-null  object
 3   video_id           33762 non-null  object
 4   video_title        33762 non-null  object
 5   video_description  33762 non-null  object
 6   comment_text       33762 non-null  object
dtypes: object(7)
memory usage: 1.8+ MB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42706 entries, 0 to 42705
Data columns (total 7 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   channel_id         42706 non-null  object
 1   channel_title      42706 non-null  object
 2   affiliation        42706 non-null  object
 3   video_id           42706 non-null  object
 4   video_title        42706 non-

In [168]:
model_sizes = np.logspace(2, 5, 4, base=5).astype(int).tolist()
model_sizes

[25, 125, 625, 3125]

### Lets begin by creating a function which can iteratively reduce a set of comments down to size of the largest desired model.

In [140]:
def get_flagged_lines(msg):
    idx = msg.index('Flagged lines: ') + len('Flagged lines: ')
    temp = msg[idx:].split(', ')
    try:
        if temp[-1].index(' ') > -1:
            temp[-1] = temp[-1][:temp[-1].index(' ')]
    except ValueError:
        pass
    return set([int(x) for x in temp])

In [141]:
def create_training_file(df, affil, final=False):
    size = df.shape[0]
    if final:
        if affil == 'l':
            path = f'../data/final_ft_datasets/progressive_size_{size}_train.jsonl'
        else:
            path = f'../data/final_ft_datasets/conservative_size_{size}_train.jsonl'
    else:
        if affil == 'l':
            path = f'../data/cleaned/progressive_size_{size}_train.jsonl'
        else:
            path = f'../data/cleaned/conservative_size_{size}_train.jsonl'
        
    ft_train = open(path, mode='w')

    for i in range(size):
        row_dic = format_comment_for_finetuning(df.iloc[i,:])
        ft_train.write(row_dic)
        ft_train.write('\n')
    
    print(f"Successfully wrote size {affil} {size} training file.")
    
    return path


In [164]:
def reduce_finetuning_set(df, backup_df, desired_size, affil, flagged_lines=[], reduce=False, skip = False, edge_case=False):
    if not skip:
        ftj_obj = {"status":"skipped"}
        if not flagged_lines and not reduce:
            if affil == 'l':
                affil_label = "progressive"
            else:
                affil_label = "conservative"
                
            #Create initial File  
            size = df.shape[0]
            print(f"Reducing dataframe with {size} examples.")
            path = create_training_file(df, affil)
            upload_response = openai.File.create(
                file=open(path, "rb"),
                purpose='fine-tune',
                user_provided_filename=f'{affil_label}_size_{size}_training_examples'
            )
        
            print("Creating fine-tuning job.")
            #Attempt to fine_tune on entire list 
            file_id = upload_response['id']
            ftj_response = openai.FineTuningJob.create(
                training_file=file_id,
                model="gpt-3.5-turbo"
            )

            base_cases = ["failed", "succeeded", "cancelled"]
            loop_count = 1
            while openai.FineTuningJob.retrieve(ftj_response['id'])['status'] not in base_cases:
                time.sleep(5)
                if loop_count % 60 == 0:
                    print(f"Waiting for finetuning job!\nTime: {loop_count*5//60} minutes")
                loop_count+=1
            
            ftj_obj = openai.FineTuningJob.retrieve(ftj_response['id'])  
            
        if flagged_lines or ftj_obj['status'] == 'failed':
            print("Finetuning job failed. Recursing.")
            if not flagged_lines:
                msg = ftj_obj['error']['message']
                flagged_lines = get_flagged_lines(msg)
            reduced_df = df.drop(flagged_lines)
            if reduced_df.shape[0] < desired_size:
                added_count = desired_size-reduced_df.shape[0]
                print(f"Added {added_count} examples from backup dataframe.")
                reduced_df = pd.concat([reduced_df, backup_df.sample(n=added_count)])
                edge_case = True
            else:
                edge_case = False
            reduced_df.reset_index(drop=True, inplace=True)
            return reduce_finetuning_set(reduced_df, backup_df, desired_size, affil, edge_case=edge_case)
        else:
            if edge_case:
                return ftj_obj, df
            else:
                print("Finetuning job succeeded.")
                if size > desired_size:
                    print(f"Succeeded with dataframe of {size}. Reducing to {desired_size}.")
                    reduced_df = df.sample(n=desired_size)
                    final_path = create_training_file(reduced_df,affil,final=True)
                else:
                    final_path = create_training_file(df,affil,final=True)    
                    
                final_upload_response = openai.File.create(
                    file=open(final_path, "rb"),
                    purpose='fine-tune',
                    user_provided_filename=f'{affil_label}_size_{desired_size}_training_examples'
                )
            
                final_file_id = final_upload_response['id']
                final_ftj_response = openai.FineTuningJob.create(
                    training_file=final_file_id,
                    model="gpt-3.5-turbo"
                )
                
                print(f"Final {affil_label} fine-tuning file of size {desired_size} created.")
                return final_ftj_response, reduced_df
    else:
        size = df.shape[0]
        if affil == 'l':
            affil_label = "progressive"
        else:
            affil_label = "conservative"
        if size > desired_size:
            print(f"Reducing dataframe of {size} to {desired_size}.")
            reduced_df = df.sample(n=desired_size)
            final_path = create_training_file(reduced_df,affil,final=True)
        else:
            final_path = create_training_file(df,affil,final=True)    
            
        final_upload_response = openai.File.create(
            file=open(final_path, "rb"),
            purpose='fine-tune',
            user_provided_filename=f'{affil_label}_size_{desired_size}_training_examples'
        )
    
        final_file_id = final_upload_response['id']
        final_ftj_response = openai.FineTuningJob.create(
            training_file=final_file_id,
            model="gpt-3.5-turbo"
        )
        
        print(f"Final {affil_label} fine-tuning file of size {desired_size} created.")
        return final_ftj_response, reduced_df

In [165]:
model_sizes = np.logspace(2, 6, 4, base=5).astype(int).tolist()
model_sizes

[25, 213, 1827, 15625]

# OOOOOOOOOOOOOOPS 

model_sizes = np.logspace(2, 6, 5, base=5).astype(int).tolist()

In [144]:
def finetune_affliated_models(comments, affil, model_sizes):
    largest_model = model_sizes[-1]
    seed = 123
    old_df = comments
    new_df = comments.sample(n=int(largest_model+(.25*largest_model)), random_state=seed).reset_index(drop=True)
    models = {}
    skip = False
    for i in range(len(model_sizes)-1, -1, -1):
        model_size= model_sizes[i]
        response, temp_df = reduce_finetuning_set(new_df, old_df, model_size, affil, skip=skip)
        while openai.FineTuningJob.retrieve(response['id'])['status'] != 'succeeded':
                time.sleep(5)
        models[model_size] = response['id']
        old_df = new_df.reset_index(drop=True)
        new_df = temp_df.reset_index(drop=True)
        skip = True
    return models
        

Nice! Now let's create our models!

In [163]:
finetune_affliated_models(right_comments, 'r', model_sizes[:2])


Reducing dataframe with 156 examples.
Successfully wrote size r 156 training file.
Creating fine-tuning job.
Waiting for finetuning job!
Time: 5 minutes
Waiting for finetuning job!
Time: 10 minutes
Waiting for finetuning job!
Time: 15 minutes
Finetuning job succeeded.
Succeeded with dataframe of 156. Reducing to 125.
Successfully wrote size r 125 training file.
Final conservative fine-tuning file of size 125 created.
Reducing dataframe of 125 to 25.
Successfully wrote size r 25 training file.
Final conservative fine-tuning file of size 25 created.


{125: 'ftjob-dUQD3xBBWUQ33mA1AcTRUxSr', 25: 'ftjob-q5UHn926xz9g8wfdCPjFuK7U'}